In [1]:
import numpy as np

In [2]:
# Import phi from train data set
phi = np.loadtxt('C:/Users/Sam/Desktop/Guvi/DW22/Task 6/data/train.csv', dtype='float', delimiter=',', skiprows=1, usecols=tuple(range(1, 14)))

In [3]:
# Import phi_test from test data set
phi_test = np.loadtxt('C:/Users/Sam/Desktop/Guvi/DW22/Task 6/data/test.csv', dtype='float', delimiter=',', skiprows=1, usecols=tuple(range(1, 14)))

In [4]:
#Import y from train dataset using the loadtxt function
y = np.loadtxt('C:/Users/Sam/Desktop/Guvi/DW22/Task 6/data/train.csv', dtype='float', delimiter=',', skiprows=1, usecols=14, ndmin=2)

In [5]:
#Concatenate coloumn of 1s to right of phi and phi_test
phi_test = np.concatenate((phi_test, np.ones((105, 1))), axis=1)
phi = np.concatenate((phi, np.ones((400, 1))), axis=1)

In [6]:
#Apply min max scaling on each coloumn of phi and phi_test

for i in range(0, 13):
    col_max = max(phi[:, i])
    col_min = min(phi[:, i])
    phi[:, i] = (phi[:, i] - col_min) / (col_max - col_min)
    phi_test[:, i] = (phi_test[:, i] - col_min) / (col_max - col_min)

In [7]:
#Apply log scaling on y
y = np.log(y)

In [8]:
#Define a function to calculate change in error function based on phi, w and p norm
def change_func(p, phi, w):
    if p == 2:
        change = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)))
    if p < 2 and p > 1:
        change = (2 * (np.dot(np.dot(np.transpose(phi), phi), w) -
                       np.dot(np.transpose(phi), y)) +
                  lambd * p * np.power(np.absolute(w), (p - 1)) * np.sign(w))
    return change

In [9]:
#Make a dictionary containing filenames as keys and p as values
filenames = {'output.csv': 2.0,
             'output_p1.csv': 1.75,
             'output_p2.csv': 1.5,
             'output_p3.csv': 1.3
             }

In [10]:
#For each item in this dictionary:

for (fname, p) in filenames.items():
    w = np.zeros((14, 1))   #Set the w to all 0s
    lambd = 0.2             #Hyperparameter lambda value
    t = 0.00012             #Max step size
    
    w_new = w - t * change_func(p, phi, w) #Calculate new value of w
    
    #Repeat steps until error between consecutive w's is less than threshold
    i = 0
    while(np.linalg.norm(w_new - w) > 10 ** -10):
        w = w_new
        w_new = w - t * change_func(p, phi, w)
        i = i + 1
        
    #Load values of id from test data file
    id_test = np.loadtxt('C:/Users/Sam/Desktop/Guvi/DW22/Task 6/data/test.csv', dtype='int', delimiter=',',
                         skiprows=1, usecols=0, ndmin=2)

    #Calculate y for test data using phi test and applying inverse log
    y_test = np.exp(np.dot(phi_test, w_new))
    
    #Save the ids and y according to filename from dictionary
    np.savetxt(fname, np.concatenate((id_test, y_test), axis=1), delimiter=',', fmt=['%d', '%f'], header='ID,MEDV', comments='')